In [1]:
import os
import csv
import json
import re
import time
import datetime
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
def get_crypto_info(i):
    api_url = 'https://api.coinmarketcap.com/v2/ticker/?start='+str(i)
    response = requests.get(api_url)

    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [3]:
def generate_cmc_api(i):
    with open('cmc_api.csv', 'w', encoding='utf-8', newline='') as csvfile:
        cmc_api_writer = csv.writer(csvfile)
        cmc_api_writer.writerow(['circulating_supply', 'ID', 'last_updated',\
                                 'max_supply', 'name', 'market_cap',\
                                 'percent_change_1h', 'percent_change_24h', 'percent_change_7d',\
                                 'price', 'volume_24h', 'rank',\
                                 'symbol', 'total_supply', 'website_slug'])
        while get_crypto_info(i):
            for crypto in get_crypto_info(i)['data']:
                c = get_crypto_info(i)['data'][crypto]
                circulating_supply = c['circulating_supply']
                ID = c['id']
                last_updated = c['last_updated']
                max_supply = c['max_supply']
                name = c['name']
                q = c['quotes']['USD']
                market_cap = q['market_cap']
                percent_change_1h = q['percent_change_1h']
                percent_change_24h = q['percent_change_24h']
                percent_change_7d = q['percent_change_7d']
                price = q['price']
                volume_24h = q['volume_24h']
                rank = c['rank']
                symbol = c['symbol']
                total_supply = c['total_supply']
                website_slug = c['website_slug']
                cmc_api_writer.writerow([circulating_supply, ID, last_updated,\
                                         max_supply, name, market_cap,\
                                         percent_change_1h, percent_change_24h, percent_change_7d,\
                                         price, volume_24h, rank,\
                                         symbol, total_supply, website_slug])
            i += 100

In [4]:
def beautify_url(seed, url):
    if url:
        if url[:4] != 'http':
            if url[0] == '/':
                if seed[-1] == '/':
                    url = seed + url[1:]
                else:
                    url = seed + url
            else:
                url = seed + '/' + url
    return url

In [5]:
def generate_cmc_web():
    cmc_url = 'https://coinmarketcap.com/all/views/all/'
    response = requests.get(cmc_url).text
    soup = BeautifulSoup(response, 'lxml')
    trs = soup.find_all('tr')
    with open('cmc_web.csv', 'w', encoding='utf-8', newline='') as csvfile:
        cmc_web_writer = csv.writer(csvfile)
        cmc_web_writer.writerow(['name', 'symbol', 'website', 'github', 'careers'])
        for tr in trs[1:]:
            symbol = tr.find('span', {'class': 'currency-symbol'}).text
            name = tr.find('a', {'class': 'currency-name-container'}).text

            coin_url = tr.find('a').get('href')
            coin_url = 'https://coinmarketcap.com' + coin_url
            response = requests.get(coin_url).text
            soup = BeautifulSoup(response, 'lxml')


            website = soup.find('a', text='Website').get('href') if soup.find('a', text='Website') else None
            github = soup.find('a', text='Source Code').get('href') if soup.find('a', text='Source Code') else None
            careers = None

            if website:
                try:
                    response2 = requests.get(website).text
                    soup2 = BeautifulSoup(response2, 'lxml')

                    careers = soup2.find('a', {'href': re.compile(r'(careers|jobs)')}).get('href') if soup2.find('a', {'href': re.compile(r'(careers|jobs)')}) else None
                    careers = beautify_url(website, careers)

                    if not github:
                        github = soup2.find('a', {'href': re.compile(r'github')}).get('href') if soup2.find('a', {'href': re.compile(r'github')}) else None
                        github = beautify_url(website, github)
                except Exception:
                    pass
            cmc_web_writer.writerow([name, symbol, website, github, careers])
            time.sleep(0.5)

In [6]:
def generate_cmc_gems(i):
    with open('cmc_gems.csv', 'w', encoding='utf-8', newline='') as csvfile:
        cmc_gems_writer = csv.writer(csvfile)
        cmc_gems_writer.writerow(['circulating_supply', 'ID', 'last_updated',\
                                 'max_supply', 'name', 'market_cap',\
                                 'percent_change_1h', 'percent_change_24h', 'percent_change_7d',\
                                 'price', 'volume_24h', 'rank',\
                                 'symbol', 'total_supply', 'website_slug', 'website', 'github'])
        while get_crypto_info(i):
            for crypto in get_crypto_info(i)['data']:
                c = get_crypto_info(i)['data'][crypto]
                q = c['quotes']['USD']
                market_cap = q['market_cap']
                volume_24h = q['volume_24h']
                if volume_24h is not None:
                    if market_cap is None and volume_24h > 500000:
                        circulating_supply = c['circulating_supply']
                        ID = c['id']
                        last_updated = c['last_updated']
                        max_supply = c['max_supply']
                        name = c['name']
                        percent_change_1h = q['percent_change_1h']
                        percent_change_24h = q['percent_change_24h']
                        percent_change_7d = q['percent_change_7d']
                        price = q['price']
                        rank = c['rank']
                        symbol = c['symbol']
                        total_supply = c['total_supply']
                        website_slug = c['website_slug']
                        
                        # Extract website and github
                        cmc_gem_url = 'https://coinmarketcap.com/currencies/' + website_slug
                        response = requests.get(cmc_gem_url).text
                        soup = BeautifulSoup(response, 'lxml')
                        website = soup.find('a', text='Website').get('href') if soup.find('a', text='Website') else None
                        github = soup.find('a', text='Source Code').get('href') if soup.find('a', text='Source Code') else None
                        
                        cmc_gems_writer.writerow([circulating_supply, ID, last_updated,\
                                                 max_supply, name, market_cap,\
                                                 percent_change_1h, percent_change_24h, percent_change_7d,\
                                                 price, volume_24h, rank,\
                                                 symbol, total_supply, website_slug,\
                                                 website, github])
            i += 100

In [15]:
def generate_cmc_gems_daily(i, gems_names):
    filename = './cmc_gems_daily/' + datetime.datetime.today().strftime('%Y%m%d') + '.csv'
    with open(filename, 'w', encoding='utf-8', newline='') as csvfile:
        cmc_gems_daily_writer = csv.writer(csvfile)
        cmc_gems_daily_writer.writerow(['circulating_supply', 'ID', 'last_updated',\
                                 'max_supply', 'name', 'market_cap',\
                                 'percent_change_1h', 'percent_change_24h', 'percent_change_7d',\
                                 'price', 'volume_24h', 'rank',\
                                 'symbol', 'total_supply', 'website_slug', 'website', 'github'])
        start_ranks = [i]
        with open(r'cmc_gems.csv', 'a') as f:
            cmc_gems_writer = csv.writer(f)
            cmc_gems_writer.writerow([])
            while get_crypto_info(i):
                for crypto in get_crypto_info(i)['data']:
                    c = get_crypto_info(i)['data'][crypto]
                    q = c['quotes']['USD']
                    market_cap = q['market_cap']
                    volume_24h = q['volume_24h']
                    if volume_24h is not None:
                        if market_cap is None and volume_24h > 90000 and c['name'] not in gems_names:
                            circulating_supply = c['circulating_supply']
                            ID = c['id']
                            last_updated = c['last_updated']
                            max_supply = c['max_supply']
                            name = c['name']
                            percent_change_1h = q['percent_change_1h']
                            percent_change_24h = q['percent_change_24h']
                            percent_change_7d = q['percent_change_7d']
                            price = q['price']
                            rank = c['rank']
                            symbol = c['symbol']
                            total_supply = c['total_supply']
                            website_slug = c['website_slug']

                            # Extract website and github
                            cmc_gem_url = 'https://coinmarketcap.com/currencies/' + website_slug
                            response = requests.get(cmc_gem_url).text
                            soup = BeautifulSoup(response, 'lxml')
                            website = soup.find('a', text='Website').get('href') if soup.find('a', text='Website') else None
                            github = soup.find('a', text='Source Code').get('href') if soup.find('a', text='Source Code') else None

                            results = [circulating_supply, ID, last_updated,\
                                                     max_supply, name, market_cap,\
                                                     percent_change_1h, percent_change_24h, percent_change_7d,\
                                                     price, volume_24h, rank,\
                                                     symbol, total_supply, website_slug, website, github]
                            cmc_gems_daily_writer.writerow(results)
                            cmc_gems_writer.writerow(results)

                        elif market_cap is not None:
                            start_ranks.append(c['rank']+1)

                i += 100
            f.close()
        f = open('gem_start_rank.txt','w')
        f.write(str(min(start_ranks)))
        f.close()

In [16]:
def find_cmc_gems():
    if not os.path.exists('gem_start_rank.txt'):
        f = open('gem_start_rank.txt','w')
        f.write('0')
        f.close()
        
    f = open('gem_start_rank.txt')
    start_rank = int(f.readlines()[0])
    f.close()
    
    if not os.path.exists('cmc_gems.csv'):
        generate_cmc_gems(start_rank)
    else:
        cmc_gems = pd.read_csv('cmc_gems.csv')
        generate_cmc_gems_daily(start_rank, cmc_gems['name'].values)

In [17]:
# generate_cmc_api(1)

In [18]:
# generate_cmc_web()

In [19]:
find_cmc_gems()

In [21]:
pd.read_csv('./cmc_gems_daily/' + datetime.datetime.today().strftime('%Y%m%d') + '.csv')

,circulating_supply,ID,last_updated,max_supply,name,market_cap,percent_change_1h,percent_change_24h,percent_change_7d,price,volume_24h,rank,symbol,total_supply,website_slug,website,github
0,NaN,2618,1532751298,NaN,StockChain,NaN,-0.40,2.52,11.05,0.004673,254084.0,1476,SCC,1.000000e+10,stockchain,http://stockchain.co/,https://github.com/StockChainScc
1,NaN,2959,1532751302,NaN,WeToken,NaN,20.12,55.33,NaN,0.012205,240085.0,1478,WT,6.000000e+08,wetoken,https://en.worldwifi.io/,https://github.com/World-WiFi/deal-referral-co...
2,NaN,2713,1532751299,NaN,KEY,NaN,-0.91,-1.68,-10.76,0.004088,182034.0,1483,KEY,1.000000e+11,key,https://www.bihu.com/,https://github.com/bihu-id/
3,NaN,2970,1532751302,NaN,LocalCoinSwap,NaN,-0.95,8.56,NaN,0.181706,160789.0,1487,LCS,7.273242e+07,local-coin-swap,https://www.localcoinswap.com,https://github.com/LocalCoinSwap/cryptoshares
4,NaN,2894,1532751301,NaN,OTCBTC Token,NaN,-1.05,-7.07,-17.25,0.155335,151654.0,1488,OTB,1.994709e+08,otcbtc-token,https://otcbtc.com/,NaN
5,NaN,2857,1532751301,NaN,SalPay,NaN,-0.12,3.36,-11.48,0.039223,143420.0,1489,SAL,1.000000e+08,salpay,https://www.salpay.com,NaN
6,NaN,2977,1532751302,NaN,BitRewards,NaN,-0.07,-15.25,NaN,0.004225,139624.0,1491,BIT,5.119915e+08,bitrewards,https://bitrewards.network/,https://github.com/BitRewards
7,NaN,2975,1532751302,NaN,FundToken,NaN,-0.22,14.38,NaN,0.021926,137666.0,1492,FUNDZ,7.979659e+07,fundtoken,https://fundfantasy.com/,https://github.com/FundFantasy/TokenSale
8,NaN,2940,1532751302,NaN,Sp8de,NaN,-0.07,26.82,-0.33,0.000347,128277.0,1495,SPX,8.888889e+09,sp8de,https://sp8de.com/,NaN
9,NaN,2532,1532751296,NaN,Etherecash,NaN,6.67,-0.99,-9.22,0.010459,121646.0,1496,ECH,NaN,etherecash,http://etherecash.io/,NaN
